In [44]:
import csv
import pandas as pd

In [46]:
def generate_sql_inserts(csv_file, table_name, column_mapping):
    df = pd.read_csv(csv_file)
    df.rename(columns=column_mapping, inplace=True)

    # Drop 'Unnamed: 0' column if it exists
    if "Unnamed: 0" in df.columns:
        df.drop(columns=["Unnamed: 0"], inplace=True)
    
    df = df[list(column_mapping.values())]  # Keep only relevant columns
    
    values_list = []
    
    for _, row in df.iterrows():
        values = []
        for col in column_mapping.values():
            val = row[col]
            if pd.isna(val):
                values.append("NULL")
            elif isinstance(val, str):
                values.append(f"'{val.replace("'", "''")}'")  # Escape single quotes
            elif "Percentile" in col:
                values.append(f"{float(val):.2f}")  # Format as DECIMAL(5,2)
            else:
                values.append(str(int(val)))  # Store as an integer
        
        values_list.append(f"({', '.join(values)})")
    
    sql_insert = f"INSERT INTO {table_name} ({', '.join(column_mapping.values())}) VALUES \n" + ",\n".join(values_list) + ";"
    
    return sql_insert

# Define input CSV files and table names
table_mappings = [
    ("cpsc368-project/preprocessed_health_data.csv", "HealthExpenditure", {
        "year": "Year",
        "gender": "Gender",
        "age_group": "AgeGroup",
        "total_current_dollars": "CurrentDollars",
        "total_current_dollars_per_capita": "CurrentDollarsPerCapita"
    })
]

# Generate SQL statements
sql_statements = []
for csv_file, table_name, column_mapping in table_mappings:
    sql_statements.append(generate_sql_inserts(csv_file, table_name, column_mapping))

# Run the script
csv_file_path = "cpsc368-project/preprocessed_health_data.csv"
table_name = "HealthExpenditure"
sql_script = generate_sql_inserts(csv_file_path, table_name, table_mappings[0][2])

# Write to a new SQL file
output_file = "health_expenditure_insert.sql"
with open(output_file, "w") as f:
    f.write("\n\n".join(sql_statements))

In [55]:
def generate_sql_inserts_wait_times(csv_file, table_name):
    df = pd.read_csv(csv_file)

    # Rename columns to match the database schema
    column_mapping = {
        "Year": "Year",
        "HEALTH_AUTHORITY": "HealthAuthority",
        "HOSPITAL_NAME": "HospitalName",
        "CATEGORY": "Category",
        "WAITING": "Waiting",
        "COMPLETED": "Completed",
        "COMPLETED_50TH_PERCENTILE": "Completed_50th_Percentile",
        "COMPLETED_90TH_PERCENTILE": "Completed_90th_Percentile"
    }

    df.rename(columns=column_mapping, inplace=True)

    # Drop 'Unnamed: 0' column if it exists
    if "Unnamed: 0" in df.columns:
        df.drop(columns=["Unnamed: 0"], inplace=True)

    # Select only the required columns
    df = df[list(column_mapping.values())]  

    values_list = []

    for _, row in df.iterrows():
        values = []
        for col in column_mapping.values():
            val = row[col]

            if pd.isna(val):
                values.append("NULL")
            elif isinstance(val, str):
                values.append(f"""'{val.replace("'", "''")}'""")  # Escape single quotes properly
            elif "Percentile" in col:
                values.append(f"{float(val):.2f}")  # Format as DECIMAL(5,2)
            else:
                values.append(str(int(val)))  # Store as an integer

        values_list.append(f"({', '.join(values)})")

    sql_insert = f"INSERT INTO {table_name} ({', '.join(column_mapping.values())}) VALUES \n" + ",\n".join(values_list) + ";"

    return sql_insert

# Define file path and table name
csv_file_path_wait_times = "cpsc368-project/preprocessed_surgical_wait_times.csv"
table_name = "HospitalWaitTimes"

# Generate SQL script
sql_script = generate_sql_inserts_wait_times(csv_file_path_wait_times, table_name)

# Write to a new SQL file
output_file = "surgical_wait_times_insert.sql"
with open(output_file, "w") as f:
    f.write(sql_script)